# Elementary Recipe for Quantum Circuit Unoptimization

Code that implements the elementary recipe from "Quantum Circuit Unoptimization" ([arXiv:2311.03805](https://arxiv.org/pdf/2311.03805)).

## Preliminaries

In [1]:
import numpy as np
import random

from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator, random_unitary

First, we need a way of generating a random circuit consisting of two-qubit gates.

In [2]:
def random_two_qubit_circuit(num_qubits: int, depth: int) -> QuantumCircuit:
    """Generate a random quantum circuit with two-qubit gates.

    Args:
        num_qubits: The number of qubits in the circuit.
        depth: The number of layers (depth) of two-qubit gates.

    Returns:
        The generated random circuit.
    """
    qc = QuantumCircuit(num_qubits)

    # Map gate names to their corresponding methods
    gate_map = {
        "cx": qc.cx,
        "cz": qc.cz,
        "swap": qc.swap,
        "iswap": qc.iswap
    }

    for _ in range(depth):
        qubit1, qubit2 = random.sample(range(num_qubits), 2)
        gate = random.choice(list(gate_map.keys()))
        gate_map[gate](qubit1, qubit2)

    return qc

As an example, here is randomly generated 5-qubit and depth-10 circuit consisting of two-qubit gates 

In [3]:
num_qubits = 5
depth = 10
random_circuit = random_two_qubit_circuit(num_qubits, depth)

print(random_circuit)

                                    ┌───┐   ┌────────┐
q_0: ────────────■──────────────────┤ X ├───┤1       ├
     ┌────────┐  │                  └─┬─┘   │        │
q_1: ┤0       ├──■─────────────■──■───┼───X─┤  Iswap ├
     │        │     ┌────────┐ │  │   │   │ │        │
q_2: ┤  Iswap ├──■──┤1       ├─■──┼───┼───┼─┤0       ├
     │        │┌─┴─┐│  Iswap │    │   │   │ └────────┘
q_3: ┤1       ├┤ X ├┤0       ├─■──┼───■───┼───────────
     └────────┘└───┘└────────┘ │  │       │           
q_4: ──────────────────────────■──■───────X───────────
                                                      


## Elementary recipe

The elementary recipe (ER) for quantum circuit unoptimization is given by Figure-1 in [arXiv:2311.03805](https://arxiv.org/pdf/2311.03805). The ER contains the following steps (applied in the following order):

1. Gate insertion
2. Gate swapping
3. Gate decomposition
4. Gate synthesis

### Gate insertion

In [6]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import random_unitary, Operator
import numpy as np

def gate_insert(qc: QuantumCircuit) -> QuantumCircuit:
    """Insert a two-qubit gate A and a modified version of its Hermitian conjugate between two gates B1 and B2 that share a common qubit.

    Args:
        qc: The input quantum circuit.

    Returns:
        The modified quantum circuit with A and modified A† inserted.
    """
    # Collect all two-qubit gates with their indices and qubits
    two_qubit_gates = []
    for idx, instruction in enumerate(qc.data):
        instr = instruction.operation
        qargs = instruction.qubits
        cargs = instruction.clbits
        if len(qargs) == 2:
            qubit_indices = [qc.find_bit(qarg).index for qarg in qargs]
            two_qubit_gates.append({'index': idx, 'qubits': qubit_indices, 'gate': instr})

    # Find a pair of gates that share a common qubit
    found_pair = False
    for i in range(len(two_qubit_gates)):
        for j in range(i+1, len(two_qubit_gates)):
            qubits_i = set(two_qubit_gates[i]['qubits'])
            qubits_j = set(two_qubit_gates[j]['qubits'])
            common_qubits = qubits_i & qubits_j
            if len(common_qubits) == 1:
                B1_idx = two_qubit_gates[i]['index']
                B2_idx = two_qubit_gates[j]['index']
                B1_qubits = two_qubit_gates[i]['qubits']
                B2_qubits = two_qubit_gates[j]['qubits']
                B1_gate = two_qubit_gates[i]['gate']
                shared_qubit = list(common_qubits)[0]
                found_pair = True
                break
        if found_pair:
            break

    if not found_pair:
        raise ValueError("No pair of two-qubit gates sharing a common qubit found.")

    # Choose a third qubit not involved in B1
    all_qubits = set(range(qc.num_qubits))
    other_qubits = list(all_qubits - set(B1_qubits))
    if not other_qubits:
        raise ValueError("Not enough qubits to perform the operation.")
    third_qubit = other_qubits[0]

    # Map indices back to qubits
    qubit_map = {qc.find_bit(q).index: q for q in qc.qubits}

    # Get the qubit objects
    q1 = qubit_map[B1_qubits[0]]
    q2 = qubit_map[B1_qubits[1]]
    q3 = qubit_map[third_qubit]

    # Generate a random two-qubit unitary A
    A = random_unitary(4)
    A_dag = A.adjoint()

    # Get the unitary matrix of B1
    B1_operator = Operator(B1_gate)
    U_B1 = B1_operator.data

    # Compute the three-qubit unitary \widetilde{A^\dagger}
    I = np.eye(2)  # Identity for a single qubit

    # Compute B1 ⊗ I
    B1_tensor_I = np.kron(U_B1, I)

    # Compute (B1 ⊗ I)^\dagger
    B1_tensor_I_dag = B1_tensor_I.conj().T

    # Compute I ⊗ A^\dagger
    I_tensor_A_dag = np.kron(I, A_dag.data)

    # Compute \widetilde{A^\dagger}
    A_dag_tilde_data = B1_tensor_I_dag @ I_tensor_A_dag @ B1_tensor_I
    A_dag_tilde = Operator(A_dag_tilde_data)

    # Create a new circuit
    new_qc = QuantumCircuit(qc.num_qubits)

    # Copy the gates up to and including B1
    for instruction in qc.data[:B1_idx+1]:
        instr = instruction.operation
        qargs = instruction.qubits
        cargs = instruction.clbits
        new_qc.append(instr, qargs, cargs)

    # Insert A on qubits [shared_qubit, third_qubit]
    qubits_for_A = [qubit_map[shared_qubit], qubit_map[third_qubit]]
    new_qc.unitary(A, qubits_for_A, label="A")

    # Insert \widetilde{A^\dagger} on qubits [B1_qubits[0], B1_qubits[1], third_qubit]
    qubits_for_A_dag_tilde = [q1, q2, q3]
    new_qc.unitary(A_dag_tilde, qubits_for_A_dag_tilde, label="A_dag_tilde")

    # Copy the remaining gates
    for instruction in qc.data[B1_idx+1:]:
        instr = instruction.operation
        qargs = instruction.qubits
        cargs = instruction.clbits
        new_qc.append(instr, qargs, cargs)

    return new_qc

In [7]:
# Example usage
qc = random_two_qubit_circuit(4, 5)
new_qc = gate_insert(qc)
print(new_qc)

     ┌───┐      ┌──────────────┐          ┌────────┐   ┌───┐
q_0: ┤ X ├──────┤1             ├──────────┤0       ├───┤ X ├
     └─┬─┘┌────┐│              │          │        │   └─┬─┘
q_1: ──┼──┤1   ├┤2             ├──────────┤        ├─X───┼──
       │  │    ││  A_dag_tilde │┌────────┐│  Iswap │ │   │  
q_2: ──┼──┤  A ├┤              ├┤0       ├┤        ├─X───┼──
       │  │    ││              ││  Iswap ││        │     │  
q_3: ──■──┤0   ├┤0             ├┤1       ├┤1       ├─────■──
          └────┘└──────────────┘└────────┘└────────┘        


### Gate swapping

In [ ]:
def gate_swap():
    pass

### Gate decomposition

In [ ]:
def gate_decompose():
    pass

### Gate synthesis

In [ ]:
def gate_synthesize():
    pass